In [1]:
import atexit
import datetime
import numpy as np
import os
import torch
import tensorboardX

from prob_mbrl import utils, models, algorithms, envs
from functools import partial
torch.set_flush_denormal(True)
torch.set_num_threads(1)
torch.manual_seed(0)
np.random.seed(0)

In [2]:
# parameters
n_rnd = 4
pred_H = 25
control_H = 40
N_particles = 100
pol_opt_iters = 1000
dyn_opt_iters = 2000
ps_iters = 100
dyn_components = 1
dyn_shape = [200] * 2
pol_shape = [200] * 2
use_cuda = False
learn_reward = False
keep_best = False

# initialize environment
# env = envs.Pendulum() # this works better with learning the reward function
env = envs.Cartpole()

results_filename = os.path.expanduser(
    "~/.prob_mbrl/results_%s_%s.pth.tar" %
    (env.__class__.__name__,
     datetime.datetime.now().strftime("%Y%m%d%H%M%S.%f")))
D = env.observation_space.shape[0]
U = env.action_space.shape[0]
maxU = env.action_space.high
minU = env.action_space.low

# initialize reward/cost function
if learn_reward or env.reward_func is None:
    reward_func = None
else:
    reward_func = env.reward_func

# intialize to max episode steps if available
if hasattr(env, 'spec'):
    if hasattr(env.spec, 'max_episode_steps'):
        control_H = env.spec.max_episode_steps
initial_experience = control_H * n_rnd

# initialize dynamics model
dynE = 2 * (D + 1) if learn_reward else 2 * D
if dyn_components > 1:
    output_density = models.GaussianMixtureDensity(dynE / 2,
                                                   dyn_components)
    dynE = (dynE + 1) * dyn_components + 1
else:
    output_density = models.DiagGaussianDensity(dynE / 2)

dyn_model = models.mlp(D + U,
                       dynE,
                       dyn_shape,
                       dropout_layers=[
                           models.modules.CDropout(0.25, 0.1)
                           for i in range(len(dyn_shape))
                       ],
                       nonlin=torch.nn.ReLU)
dyn = models.DynamicsModel(dyn_model,
                           reward_func=reward_func,
                           output_density=output_density).float()

# initalize policy
pol_model = models.mlp(D,
                       2 * U,
                       pol_shape,
                       dropout_layers=[
                           models.modules.BDropout(0.1)
                           for i in range(len(pol_shape))
                       ],
                       nonlin=torch.nn.ReLU,
                       output_nonlin=partial(models.DiagGaussianDensity,
                                             U))

pol = models.Policy(pol_model, maxU, minU).float()
print(dyn)
print(pol)

# initalize experience dataset
exp = utils.ExperienceDataset()

# initialize dynamics optimizer
opt1 = torch.optim.Adam(dyn.parameters(), 1e-4)

# initialize policy optimizer
opt2 = torch.optim.Adam(pol.parameters(), 1e-4)

DynamicsModel(
  (model): BSequential(
    (fc0): Linear(in_features=6, out_features=200, bias=True)
    (nonlin0): ReLU()
    (drop0): CDropout(rate=0.25, temperature=0.10000000149011612, regularizer_scale=0.5)
    (fc1): Linear(in_features=200, out_features=200, bias=True)
    (nonlin1): ReLU()
    (drop1): CDropout(rate=0.25, temperature=0.10000000149011612, regularizer_scale=0.5)
    (fc_out): Linear(in_features=200, out_features=10, bias=True)
  )
  (output_density): DiagGaussianDensity(output_dims=5)
  (reward_func): CartpoleReward()
)
Policy(
  (model): BSequential(
    (fc0): Linear(in_features=5, out_features=200, bias=True)
    (nonlin0): ReLU()
    (drop0): BDropout(rate=0.10000000149011612, regularizer_scale=0.5)
    (fc1): Linear(in_features=200, out_features=200, bias=True)
    (nonlin1): ReLU()
    (drop1): BDropout(rate=0.10000000149011612, regularizer_scale=0.5)
    (fc_out): Linear(in_features=200, out_features=2, bias=True)
    (fc_nonlin): DiagGaussianDensity(output

In [ ]:
%matplotlib qt5
if use_cuda and torch.cuda.is_available():
    dyn = dyn.cuda()
    pol = pol.cuda()

writer = tensorboardX.SummaryWriter()

# callbacks
def on_close():
    writer.close()

atexit.register(on_close)

# initial experience data collection
scale = maxU - minU
bias = minU
rnd = lambda x, t: (scale * np.random.rand(U, ) + bias)  # noqa: E731
while exp.n_samples() < initial_experience:
    ret = utils.apply_controller(
        env,
        rnd,
        control_H,
        realtime=False,
        stop_when_done=False,
        callback=lambda *args, **kwargs: env.render())
    params_ = [p.clone() for p in list(pol.parameters())]
    exp.append_episode(*ret, policy_params=params_)
    exp.save(results_filename)

# policy learning loop
for ps_it in range(ps_iters):
    if ps_it > 0 or exp.n_samples() == 0:
        # apply policy
        ret = utils.apply_controller(
            env,
            pol,
            control_H,
            callback=lambda *args, **kwargs: env.render(),
            realtime=False, stop_when_done=False)
        params_ = [p.clone() for p in list(pol.parameters())]
        exp.append_episode(*ret, policy_params=params_)
        exp.save(results_filename)

    # train dynamics
    X, Y = exp.get_dynmodel_dataset(deltas=True, return_costs=learn_reward)
    dyn.set_dataset(X.to(dyn.X.device).float(), Y.to(dyn.X.device).float())
    utils.train_regressor(dyn,
                          dyn_opt_iters,
                          N_particles,
                          True,
                          opt1,
                          log_likelihood=dyn.output_density.log_prob,
                          summary_writer=writer,
                          summary_scope='model_learning/episode_%d' %
                          ps_it)

    # sample initial states for policy optimization
    x0 = exp.sample_states(N_particles,
                           timestep=0).to(dyn.X.device).float()
    x0 = x0 + 1e-1 * torch.randn_like(x0)
    x0 = x0.detach()

    utils.plot_rollout(x0, dyn, pol, control_H)

    # train policy
    def on_iteration(i, loss, states, actions, rewards, discount):
        writer.add_scalar('mc_pilco/episode_%d/training loss' % ps_it,
                          loss, i)
        if i % 100 == 0:
            '''
            states = states.transpose(0, 1).cpu().detach().numpy()
            actions = actions.transpose(0, 1).cpu().detach().numpy()
            rewards = rewards.transpose(0, 1).cpu().detach().numpy()
            utils.plot_trajectories(states,
                                    actions,
                                    rewards,
                                    plot_samples=True)
            '''
            writer.flush()

    print("Policy search iteration %d" % (ps_it + 1))
    algorithms.mc_pilco(x0,
                        dyn,
                        pol,
                        pred_H,
                        opt2,
                        exp,
                        pol_opt_iters,
                        pegasus=True,
                        mm_states=True,
                        mm_rewards=True,
                        maximize=True,
                        clip_grad=1.0,
                        on_iteration=on_iteration,
                        step_idx_to_sample=0,
                        init_state_noise=1e-1 * x0.std(0))
    utils.plot_rollout(x0, dyn, pol, control_H)
    writer.add_scalar('robot/evaluation_loss',
                      torch.tensor(ret[2]).sum(), ps_it + 1)


apply_controller Starting run
apply_controller Running for 4.000000 seconds
apply_controller Done after [40] steps. Stopping robot. Value of run [0.009175]
apply_controller Starting run
apply_controller Running for 4.000000 seconds
apply_controller Done after [40] steps. Stopping robot. Value of run [0.572404]
apply_controller Starting run
apply_controller Running for 4.000000 seconds
apply_controller Done after [40] steps. Stopping robot. Value of run [0.006426]
apply_controller Starting run
apply_controller Running for 4.000000 seconds
apply_controller Done after [40] steps. Stopping robot. Value of run [0.025665]


log-likelihood of data: -7.943673:   1%|          | 14/2000 [00:00<00:14, 135.28it/s]

train_regressor > Dataset size [156]


  0%|          | 0/1000 [00:00<?, ?it/s]

Policy search iteration 1


...

Pred. Cumm. rewards: 18.055079 [25]: 100%|██████████| 1000/1000 [01:30<00:00, 11.02it/s]


apply_controller Starting run
apply_controller Running for 4.000000 seconds
apply_controller Done after [40] steps. Stopping robot. Value of run [32.969631]


log-likelihood of data: 13.322544:   1%|          | 14/2000 [00:00<00:15, 131.16it/s]

train_regressor > Dataset size [1053]


log-likelihood of data: 13.291028:  56%|█████▋    | 1129/2000 [00:08<00:06, 132.01it/s]